In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")
from src.settings.settings import Settings
from src.db.mongo.interfaces import ChatDocument
import uuid
import traceback

def create_new_chat(user_id : uuid.UUID):
    client, collection = Settings.MongoDB.get_chats_collection()
    
    new_chat = ChatDocument()
    new_chat.user_id = user_id
    new_chat.conversation_id = uuid.uuid4()
    
    new_chat_document = new_chat.chat_document_to_dict()
    try:
        collection.insert_one(new_chat_document).inserted_id
        print("Nuevo chat creado!\n", new_chat_document)
    except Exception as e:
        print(f"Error durante la conexion: {e}")
    
    finally:
        client.close()
        print("Conexion finalizada")


In [2]:
def find_chat_by_id(chat_id: uuid.UUID):
    client, collection = Settings.MongoDB.get_chats_collection()

    try:
        chat = collection.find_one({"conversation_id": str(chat_id)})
        print("Chat has been found")
        chat_document = ChatDocument.parse_dict_to_document(chat)
        
        return chat_document
    
    except Exception as e:
        print(f"Error durante la conexion: {e}")
        traceback.print_exc()
        
    finally:
        client.close()
        print("Conexion finalizada")

In [3]:
def save_to_chat(new_data: ChatDocument, conversation_id: uuid.UUID):
    client, collection = Settings.MongoDB.get_chats_collection()
    
    chat_dict = new_data.chat_document_to_dict()
    del chat_dict["user_id"]
    del chat_dict["conversation_id"]
    print(chat_dict)

    try:
        collection.update_one(
            {"conversation_id": str(conversation_id)}, 
            {"$set": chat_dict}, 
            upsert=True
        )
        print("Save to chat successfully")
        
    except Exception as e:
        print(f"Error durante la conexion: {e}")
    finally:
        client.close()
        print("Conexion finalizada")

In [ ]:
from datetime import datetime
from src.components.memory.memory_interfaces import HumanMessage, AIMessage

conversation_id = uuid.UUID("e5022553-cc15-427a-99a3-6e20ac5359f6")
chat_document = ChatDocument()

chat_document.last_user_message = HumanMessage("I want a list of measurement systems")
chat_document.last_assistant_message = AIMessage(
    uuid.UUID("95e7574b-0192-45f8-873b-746d33eba54f"),
    "Hello Lauther, how are ypu?",
    sql_response="asdsada"
)
chat_document.messages = [
    chat_document.last_user_message,
    chat_document.last_assistant_message,
]

chat_document.last_interaction = datetime.now()
chat_document.conversation_slots = "measurement_systems=database"
chat_document.current_summary = (
    "The user is introducing himself and requires a list of measurement systems"
)

chat = save_to_chat(chat_document, conversation_id)

chat

In [4]:
conversation_id = uuid.UUID("e5022553-cc15-427a-99a3-6e20ac5359f6")

chat = find_chat_by_id(conversation_id)
chat

Chat has been found
{'message_id': '181e84b1-0554-471f-b517-40e2249d4bc5', 'message': 'Hello Lauther, how are ypu?', 'message_type': 'AI', 'date_created': '2024-07-24T10:43:53.697479', 'replied_user_message_id': UUID('95e7574b-0192-45f8-873b-746d33eba54f'), 'sql_response': 'asdsada', 'dataframe': None}
{'message_id': '181e84b1-0554-471f-b517-40e2249d4bc5', 'message': 'Hello Lauther, how are ypu?', 'message_type': 'AI', 'date_created': '2024-07-24T10:43:53.697479', 'replied_user_message_id': UUID('95e7574b-0192-45f8-873b-746d33eba54f'), 'sql_response': 'asdsada', 'dataframe': None}
Conexion finalizada


ChatDocument(user_id=95e7574b-0192-45f8-873b-746d33eba54f,
conversation_id=e5022553-cc15-427a-99a3-6e20ac5359f6,
messages=[HumanMessage(
  message_id=UUID('b21245c8-5219-4b0d-ad3a-ca7e78bdcace'),
  message='I want a list of measurement systems',
  message_type='Human',
  date_created=datetime.datetime(2024, 7, 24, 11, 6, 1, 879938)
), AIMessage(
  replied_user_message_id=UUID('95e7574b-0192-45f8-873b-746d33eba54f')
  message_id=UUID('d389d587-bdff-4a7d-aedb-6f29fe5e12fd'),
  message='Hello Lauther, how are ypu?',
  message_type='AI',
  date_created=datetime.datetime(2024, 7, 24, 11, 6, 1, 879938),
  sql_response='asdsada',
  dataframe=None,
)],
last_user_message=HumanMessage(
  message_id=UUID('b17fec19-9232-491f-8d66-3cccd7ac4d6f'),
  message='I want a list of measurement systems',
  message_type='Human',
  date_created=datetime.datetime(2024, 7, 24, 11, 6, 1, 879938)
),
last_assistant_message=AIMessage(
  replied_user_message_id=UUID('95e7574b-0192-45f8-873b-746d33eba54f')
  message_